In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, to_date, split, when, col, lag, lpad, sum, mean, round, count, abs, regexp_extract, date_format, lit, expr, max 
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [8]:
spark = SparkSession.builder \
    .appName("machungkhoan") \
    .config("spark.cores.max", "2") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


In [9]:
# Đọc dữ liệu từ bảng nguồn machungkhoan
df_lichsugia = spark.read.format("iceberg").load("stock_db.datn_lichsugia")
# Đọc dữ liệu từ bảng nguồn machungkhoan
df_machungkhoan = spark.read.format("iceberg").load("stock_db.datn_machungkhoan")

In [11]:
# Convert 'ngay' to DateType
df_lichsugia = df_lichsugia.withColumn("ngay", to_date(col("ngay"), "dd/MM/yyyy"))

# Convert 'ky_du_lieu' to TimestampType
df_lichsugia = df_lichsugia.withColumn("ky_du_lieu", to_timestamp(col("ky_du_lieu"), "dd/MM/yyyy HH:mm:ss"))

# Extract numeric values from 'thaydoi' and split into change and percentage change
df_lichsugia = df_lichsugia.withColumn("thaydoi_value", regexp_extract(col("thaydoi"), r'([\d\.-]+)', 1).cast("float"))
df_lichsugia = df_lichsugia.withColumn("thaydoi_percent", regexp_extract(col("thaydoi"), r'\(([\d\.-]+)%\)', 1).cast("float"))

# Convert 'ngay' to 'dateid' in the format ddMMyyyy
df_lichsugia = df_lichsugia.withColumn("dateid", date_format(col("ngay"), "ddMMyyyy").cast("int"))


In [5]:
dim_stock_df  = df_machungkhoan.filter((col('categoryname').isNotNull()) & (col('categoryname') != ""))
df_stock_drop_duplicate = dim_stock_df.dropDuplicates(['symbol'])
# Tiếp theo, ghi dữ liệu vào bảng dim_stock
dim_stock_data = df_stock_drop_duplicate.select(
    col("symbol").alias("stocksymbol"),
    "companyname",
    "categoryid"
).distinct()

# Convert dim_stock_df to a list of stocksymbols
valid_stocksymbols = [row.stocksymbol for row in dim_stock_data.select("stocksymbol").distinct().collect()]

In [ ]:
# Select and rename columns to match the schema of 'fact_price_history'
df_fact_price_history = df_lichsugia.select(
    col("symbol").alias("stocksymbol"),
    col("dateid"),
    col("giamocua").alias("openprice").cast("decimal(18, 2)"),
    col("giadongcua").alias("closeprice").cast("decimal(18, 2)"),
    col("giacaonhat").alias("highprice").cast("decimal(18, 2)"),
    col("giathapnhat").alias("lowprice").cast("decimal(18, 2)"),
    col("khoiluongkhoplenh").alias("volume").cast("bigint")
)

In [ ]:
# Đảm bảo df_fact_price_history đã được lọc để loại bỏ các giá trị null
# Filter df_fact_price_history to only include valid stocksymbols
df_fact_price_history_filtered = df_fact_price_history.filter(col("stocksymbol").isin(valid_stocksymbols))
# df_fact_price_history_filtered = df_fact_price_history_filtered.dropna(subset=["closeprice", "volume", "highprice", "lowprice"])

df_fact_price_history_filtered = df_fact_price_history_filtered.withColumn('dateid_padded', lpad(col('dateid').cast('string'), 8, '0'))
df_fact_price_history_filtered = df_fact_price_history_filtered.withColumn('date', to_date(col('dateid_padded'), 'ddMMyyyy'))


In [ ]:
# Khung cửa sổ để truy cập giá đóng cửa và khối lượng của phiên trước đó
window_spec = Window.partitionBy("stocksymbol").orderBy(col("date").asc())

# Lấy giá đóng cửa và khối lượng của phiên trước đó
df_with_prev = df_fact_price_history_filtered.withColumn("prev_close", lag("closeprice").over(window_spec)) \
    .withColumn("prev_volume", lag("volume").over(window_spec))